# Logistic Regression Example
Using the Pima Indian Diabetes dataset
https://www.kaggle.com/uciml/pima-indians-diabetes-database

## Import Python Packages

In [ ]:
#Data Manipulation packages

import pandas as pd
import numpy as np

#Machiene Learning packages
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

#viz packages
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Loading Data

In [ ]:
col_names=['pregnant','glucose','bp','skin','insulin','bmi','pedigree','age','label']
#load in csv

pima=pd.read_csv("diabetes.csv", header=None, skiprows=[0], names=col_names)

In [ ]:
pima.head()

In [ ]:
## Data Exploration & Cleaning

In [ ]:
print('# rows = {}, # columns = {}'.format(pima.shape[0], pima.shape[1]))
pima.describe()

Are there any missing values?

In [ ]:
print(pima.isnull().sum().sum())

### Replace Missing Values 

In [ ]:
# Replace categorical missing with missing category 
#df[df.columns[df.dtypes == 'object']] = df[df.columns[df.dtypes == 'object']].fillna(value='missing')

#Replace with zero if  missing means zero
#df['xxx'].fillna(0,inplace=True)

#Replace remaining numerical missings with median
#df.fillna(df.median(), inplace=True))


### Outlier Detection & Removal

For continous numeric variables, it is worth excluding outlying data points that might otherwise skew the learning of our model later.

A standard way to do this is to use the 6-sigma idea: for a normal distribution (assumption!), 99% of the data should be within +- 3 standard deviations. So anything outside of 6 deviations should definitely be an outlier.

<img src="normal_distribution.PNG">

In [ ]:
# Find numeric values
numeric_cols = pima.columns[pima.dtypes != 'object'].tolist()

# Don't consider binary variables
for col in numeric_cols:
    if (pima[col].min() == 0) and (pima[col].max() == 1):
        numeric_cols.remove(col)

# Calculate the means and standard deviations
means = pima[numeric_cols].mean()
stddevs = pima[numeric_cols].std()

# Find out how many values are outliers
sigma = 6
outlier_percs = []
contains_outlier_any_col = np.zeros([len(pima), ]).astype(bool)
for col in numeric_cols:
    outlier = (pima[col] > means[col] + sigma * stddevs[col]) | (pima[col] < means[col] - sigma * stddevs[col])
    contains_outlier_any_col = contains_outlier_any_col | outlier
    outlier_percs.append(100 * sum(outlier) / len(outlier))
    
# Assemble results into a dataframe
outliers_df = pd.DataFrame(
    {'Column': numeric_cols, 'Outlier percentage': outlier_percs}
).sort_values(by='Outlier percentage', ascending=False)

# Total percentage of outliers
print('% of rows with an outlier = {:.2f}%'.format(100 * sum(contains_outlier_any_col) / len(contains_outlier_any_col)))

### Remove outliers

In [ ]:
pima = pima[~contains_outlier_any_col]

Validate features

In [ ]:
#is age positive?

def age_positive(ls):
    for i in ls:
        if i < 0:
            return False
    return True

age_positive(pima.age)

## Fitting the model

Going to try and predict whether someone has diabetes based on this data

### Split out target and features

In [ ]:
feature_cols = ['pregnant', 'insulin', 'bmi', 'age','glucose','bp','pedigree']
X = pima[feature_cols] # Features
y = pima.label # Target variable

### Create training and testing sets

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0,stratify=y)

### Check target distribution

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[5, 5])
counts = 100 * y_train.value_counts() / len(y_train)
sns.barplot(x=counts.index, y=counts, ax=ax)
ax.set_xlabel('Label')
ax.set_ylabel('Proportion of data (%)')
plt.show()

### Undersampling Target

In [ ]:
# Class count
count_class_0, count_class_1 = y_train.value_counts()

# Divide by class
y_class_0 = y_train[y_train== 0]
y_class_1 = y_train[y_train== 1]

#undersample 0
y_class_0_under = y_class_0.sample(count_class_1)

y_train_sampled=pd.concat([y_class_0_under, y_class_1], axis=0)

#plot undersampled target
fig, ax = plt.subplots(1, 1, figsize=[5, 5])
counts = 100 * y_train_sampled.value_counts() / len(y_train_sampled)
sns.barplot(x=counts.index, y=counts, ax=ax)
ax.set_xlabel('Label')
ax.set_ylabel('Proportion of data (%)')
plt.show()

### Model Development and Prediction

In [ ]:
# instantiate the model 
logreg = LogisticRegression()

# fit the model with train data
logreg.fit(X_train,y_train)

#apply model to test data
y_pred=logreg.predict(X_test)

## Model Evaluation

Model evaluation using Confusion Matrix

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

Visualise Confusion Matrx

In [ ]:
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

Confusion Matrix Evaluation Metrics

Precision: how often the prediction is correct

Recall: what proportion of observations have been correctly assigned

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

ROC curve

In [ ]:
y_pred_proba = logreg.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

Feature importance

In [ ]:
print(logreg.coef_)

In [ ]:
coefficients_df = pd.DataFrame({'Feature': X_test.columns, 'Coefficient': logreg.coef_[0]})
coefficients_df['Coefficient magnitude'] = coefficients_df['Coefficient'].abs()
top_10 = coefficients_df.sort_values(by='Coefficient magnitude', ascending=False)[:10]

fig, ax = plt.subplots(1, 1, figsize=[6, 7])
sns.barplot(x=top_10['Coefficient'], y=top_10['Feature'])
ax.set_title('Top 10 most important features')
plt.show()